# Домашнее задание 3

## Тема “Создание признакового пространства”

**Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.**

Установим библиотеки

In [ ]:
# !pip install spacy 

In [ ]:
# !python -m spacy download en

In [ ]:
# !pip install deeppavlov spacy

In [ ]:
# !pip install en_core_web_md

In [ ]:
# !python -m spacy download en_core_web_md

Подключим библиотеки

In [1]:
import numpy as np
import pandas as pd
import pickle

import spacy
import nltk
import en_core_web_sm
from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

Загрузим данные

In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Загрузим модель

In [3]:
with open('combine_df.pkl', 'rb') as f:
    combine_df  = pickle.load(f)

In [4]:
combine_df.head(3)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


**Задание 1.**

Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)

С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

Повторим шаги из заданий 1 и 2, используя библиотеку nltk.


In [5]:
TOP_LIMIT = 20

In [6]:
%%time
nlp = en_core_web_sm.load()
combine_df['nlp'] = combine_df.clean_tweet.progress_apply(lambda x:  nlp(x))

100%|██████████| 49159/49159 [06:06<00:00, 134.25it/s]

CPU times: user 6min 5s, sys: 2.99 s, total: 6min 8s
Wall time: 6min 7s


In [7]:
nlp.max_length = combine_df['nlp'].shape[0]

In [8]:
labels = []
for doc in combine_df['nlp'].tolist():
    for e in doc.ents:
        labels.append((e.label_, e.text))

In [9]:
ner_df = pd.DataFrame(labels, columns=['ner', 'word'])

In [10]:
ner_df.ner.value_counts().head(TOP_LIMIT)

DATE           12115
PERSON          9040
ORG             5129
GPE             4572
TIME            2225
NORP            1483
CARDINAL        1093
ORDINAL          598
FAC              305
LOC              245
EVENT            113
PRODUCT           72
LANGUAGE          54
LAW               46
WORK_OF_ART       15
QUANTITY          14
MONEY              8
PERCENT            4
Name: ner, dtype: int64

In [11]:
ner_df.loc[ner_df['ner'] == 'ORG'].word.value_counts().head(TOP_LIMIT)

tbt                119
aww                118
gop                 75
nba                 50
islam               41
nd                  40
allahsoil           34
sma                 32
ios                 30
house               30
cnn                 29
aap                 28
amazon              26
eu                  25
fed                 25
nyc                 21
fbi                 21
bogota colombia     20
congress            20
apple               20
Name: word, dtype: int64

In [12]:
ner_df.loc[ner_df['ner'] == 'GPE'].word.value_counts().head(TOP_LIMIT)

america       225
orlando       210
london        182
us            148
florida        97
uk             93
japan          79
india          78
miami          72
germany        54
paris          54
france         51
russia         51
texas          48
spain          48
new york       43
england        41
chicago        40
california     38
china          37
Name: word, dtype: int64

In [13]:
ner_df.loc[ner_df['ner'] == 'PERSON'].word.value_counts().head(TOP_LIMIT)

selfie                                                     114
hu                                                          74
trump                                                       57
tbt                                                         50
hillary                                                     42
feminismiscancer feminismisterrorism feminismmuktbharat     40
xx                                                          38
don                                                         34
christina grimmie                                           31
stas                                                        30
jo cox                                                      30
sikh temple                                                 28
donald trump                                                26
tgif ff                                                     26
god                                                         24
jesus                                                  

**Вывод:** тип NER (GPE) оказался самым популярным.

**Задание 2.**

Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.

С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?


In [14]:
string_tweet = combine_df.tweet.to_string()

In [15]:
string_tweet_tokenize = nltk.word_tokenize(string_tweet)

In [16]:
string_tweet_tags = nltk.pos_tag(string_tweet_tokenize)

In [17]:
string_tweet_chunk = nltk.ne_chunk(string_tweet_tags)

In [18]:
{(' '.join(c[0] for c in chunk), chunk.label() ) \
 for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(string_tweet))) if hasattr(chunk, 'label') }

{('___', 'ORGANIZATION'),
 ('______________', 'ORGANIZATION'),
 ('_______________________', 'ORGANIZATION'),
 ('atÂ', 'ORGANIZATION'),
 ('babalargÃ¼nÃ¼', 'ORGANIZATION'),
 ('bearishÂ', 'ORGANIZATION'),
 ('braggingÂ', 'ORGANIZATION'),
 ('brÃ¼ssel', 'ORGANIZATION'),
 ('church..and', 'GPE'),
 ('eatmorecrÃ', 'ORGANIZATION'),
 ('en-ger-land', 'GPE'),
 ('euro2016Â', 'ORGANIZATION'),
 ('euro2016Â Â', 'ORGANIZATION'),
 ('felicitÃ', 'ORGANIZATION'),
 ('firstÂ', 'ORGANIZATION'),
 ('foiinÃ', 'ORGANIZATION'),
 ('fridayÂ', 'ORGANIZATION'),
 ('fÃªte', 'ORGANIZATION'),
 ('gÃ¼mbet', 'ORGANIZATION'),
 ('ilustraciÃ³n', 'ORGANIZATION'),
 ('inc.', 'ORGANIZATION'),
 ('l.a.', 'GPE'),
 ('marschfÃ¼rjesus', 'ORGANIZATION'),
 ('mr.', 'PERSON'),
 ('mr. paris', 'PERSON'),
 ('nycÂ', 'ORGANIZATION'),
 ('oilÂ', 'ORGANIZATION'),
 ('peace/sad', 'ORGANIZATION'),
 ('st.peter', 'ORGANIZATION'),
 ('sunflowerÂ', 'ORGANIZATION'),
 ('turÂ', 'ORGANIZATION'),
 ('u.s', 'GPE'),
 ('u.s.', 'GPE'),
 ('vysoÄ\x8dina', 'ORGANIZATION')

**Вывод:** в топ вошли не только организации, но и мусор


**Задание 3.**

Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.


**Выводы:**

    - обе бибилиотеки работают не идеально. Присутствуют ошибочные детекции;
    - обе бибилиотеки есть смысл использовать только в паре с постобработкой;
    - между библиотеками есть разница при работе со словами в зависимости от регистра;
    - в топе распознавания именованных сущеностей: даты, персоны, организации;
    - какая из библиотек лучше отработала - сказать затрудняюсь.